# Bulk RNA-seq data analysis of ER and TCGA cohorts

## Setup

In [ ]:
# install packages
## warn: umap-learn might conflict with newer numba // python 3.12+
!pip install umap-learn
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install numpy
!pip install adjustText
!pip install torch
!pip install IProgress

In [17]:
# import
import importlib
import src.TCGAvsER.ML
import src.TCGAvsER.ML_viz
importlib.reload(src.TCGAvsER.ML)
importlib.reload(src.TCGAvsER.ML_viz)

from src.TCGAvsER.load import load_rnaseq_data_batch, generate_balanced_file_list, load_rnaseq_data_batch_from_list
from src.TCGAvsER.ML import LogisticRegressionML, RandomForestML, AdaBoostML, GradientBoostML, SelfSupervisedML
from src.TCGAvsER.ML import train_and_test_mls
from src.TCGAvsER.ML_viz import plot_all_metrics

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from adjustText import adjust_text
import os

# Paths
path_rnaseq_data_ER = "data/raw/ER.zip"
path_rnaseq_data_TCGA = "data/raw/TCGA.zip"

In [ ]:
# Setup R
!pip install rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.packages import importr
pandas2ri.activate()
deseq2 = importr('DESeq2')
stats = importr('stats')

In [14]:
# Unzip data
if not os.path.exists("data/decompressed/ER"):
    !mkdir -p data/decompressed/ER; unzip -q $path_rnaseq_data_ER -d data/decompressed/ER
if not os.path.exists("data/decompressed/TCGA"):
    !mkdir -p data/decompressed/TCGA; !unzip -q $path_rnaseq_data_TCGA -d data/decompressed/TCGA

In [3]:
# Plotting settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['axes.edgecolor'] = '#333333'
plt.rcParams['axes.linewidth'] = 1.2
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['grid.color'] = '#e5e5e5'
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.linewidth'] = 0.8
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['legend.frameon'] = False
plt.rcParams['legend.fontsize'] = 13
plt.rcParams['savefig.facecolor'] = 'white'
plt.rcParams['savefig.edgecolor'] = 'white'
sns.set_palette('colorblind')

## Load data

In [ ]:
## Full ER
rnaseq_data_ER = load_rnaseq_data_batch(
    root_dir="data/decompressed/ER/Transcriptome Profiling",
    verbose=True
)

In [ ]:
## Metadata ER
metadata_ER = pd.read_csv("data/metadata_ER.tsv", sep="\t")
# Get value counts and separate top 10 from others
site_counts = metadata_ER['cases.primary_site'].value_counts()
top_10 = site_counts.head(8)
other = pd.Series({'Other': site_counts[8:].sum()})

# Combine and plot
combined = pd.concat([top_10, other])
combined.plot(kind='barh', title='Primary Site Distribution', color=['#2ecc71', '#e74c3c'])
plt.xlabel('Count')
plt.ylabel('Primary Site')

In [11]:
## Balance data
balance_list ={
    "BRCA":22,
    "COAD":57,
    "ESCA":23,
    "LUAD":25,
    "OV":9,
    "PAAD":13,
    "PCPG":7
    #"SKCM":2
}

# Get balanced list
seed = 42
balanced_list = generate_balanced_file_list(
    balance_list,
    "data/decompressed/TCGA/",
    seed=seed
)

In [ ]:
rnaseq_data_TCGA = load_rnaseq_data_batch_from_list(
    balanced_list,
    verbose=True
)

In [13]:
rnaseq_data_ER_counts = rnaseq_data_ER[0].drop(columns=['gene_name', 'gene_type'])#.set_index(['gene_name', 'gene_type'])
rnaseq_data_TCGA_counts = rnaseq_data_TCGA[0].drop(columns=['gene_name', 'gene_type'])#.set_index(['gene_name', 'gene_type'])

df_combined = pd.concat([rnaseq_data_ER_counts, rnaseq_data_TCGA_counts], axis=1)

In [20]:
counts = df_combined.astype(int)

# Create metadata
Y = pd.DataFrame({
    'condition': ['ER'] * (rnaseq_data_ER_counts.shape[1]) +\
         ['TCGA'] * (rnaseq_data_TCGA_counts.shape[1])
})

Y.index = counts.columns

counts.to_csv("data/counts_combined.csv", index=False)

counts_ML = counts.copy()
counts_ML.index = rnaseq_data_ER[0].loc[:, 'gene_name']
counts_ML.to_csv("data/counts_combined_ML.csv", index=False)

Y.to_csv("data/counts_labels.csv", index=False)

In [19]:
rnaseq_data_ER_tpm = rnaseq_data_ER[3].drop(columns=['gene_name', 'gene_type'])#.set_index(['gene_name', 'gene_type'])
rnaseq_data_TCGA_tpm = rnaseq_data_TCGA[3].drop(columns=['gene_name', 'gene_type'])#.set_index(['gene_name', 'gene_type'])

df_combined_tpm = pd.concat([rnaseq_data_ER_tpm, rnaseq_data_TCGA_tpm], axis=1)
df_combined_tpm.index = rnaseq_data_ER[3].loc[:, 'gene_name']
df_combined_tpm.to_csv("data/tpm_combined.csv", index=False)

## Differential expression

In [ ]:
counts = pd.read_csv("data/counts_combined.csv")
Y = pd.read_csv("data/counts_labels.csv")

In [107]:
counts_r = pandas2ri.py2rpy(counts)
metadata_r = pandas2ri.py2rpy(Y)

In [ ]:
# Create DESeq2 dataset
dds = deseq2.DESeqDataSetFromMatrix(
    countData=counts_r,
    colData=metadata_r,
    design=Formula('~ condition')
)

# Run DESeq2
dds = deseq2.DESeq(dds)

In [119]:
# Get results
res = deseq2.results(dds)
res_df = pandas2ri.rpy2py(robjects.r['as.data.frame'](res))

gene_names = rnaseq_data_ER[0].loc[:, 'gene_name']

# Add gene names back
res_df['gene_name'] = gene_names

res_df.to_csv('data/DESeq2_results.csv', index=True)

In [ ]:
# Define significance thresholds
lfc_threshold = 2
pval_threshold = 0.001
lfc_cap = 15

# Create masks for different conditions
up_sig = (res_df['log2FoldChange'] > lfc_threshold) & (res_df['padj'] < pval_threshold)
down_sig = (res_df['log2FoldChange'] < -lfc_threshold) & (res_df['padj'] < pval_threshold)
not_sig = ~(up_sig | down_sig)

# Cap extreme log fold changes for plotting
plot_lfc = res_df['log2FoldChange'].copy()
plot_lfc[plot_lfc > lfc_cap] = lfc_cap
plot_lfc[plot_lfc < -lfc_cap] = -lfc_cap

# Plot points with different colors based on significance
plt.scatter(
    plot_lfc[not_sig],
    -np.log10(res_df.loc[not_sig, 'padj']),
    c='gray',
    alpha=0.5,
    s=20,
    label='NS'
)
plt.scatter(
    plot_lfc[up_sig],
    -np.log10(res_df.loc[up_sig, 'padj']),
    c='lightgreen',
    alpha=0.7,
    s=20,
    label='UR'
)
plt.scatter(
    plot_lfc[down_sig],
    -np.log10(res_df.loc[down_sig, 'padj']),
    c='pink',
    alpha=0.7,
    s=20,
    label='DR'
)

# Add threshold lines
plt.axhline(y=-np.log10(pval_threshold), color='r', linestyle='--', alpha=0.5)
plt.axvline(x=lfc_threshold, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=-lfc_threshold, color='r', linestyle='--', alpha=0.5)

# Get top up and down regulated genes
top_up = res_df[up_sig].nlargest(10, 'log2FoldChange')
top_down = res_df[down_sig].nsmallest(10, 'log2FoldChange')

# Prepare text annotations for repelling
texts = []
for idx, row in pd.concat([top_up, top_down]).iterrows():
    # Cap the x-coordinate for plotting
    x_coord = min(max(row['log2FoldChange'], -lfc_cap), lfc_cap)
    texts.append(plt.text(
        x_coord,
        -np.log10(row['padj']),
        row['gene_name'],
        fontsize=5,
        bbox=dict(facecolor='white', edgecolor='none', alpha=0.7)
    ))

if False:
    # Adjust text positions to avoid overlap
    adjust_text(texts, 
               arrowprops=dict(arrowstyle='->', color='gray', lw=0.01),
               expand_text=(.01, .01),
               expand_points=(.01, .01),
               force_text=(0.01, 0.01))

plt.xlabel('log2 Fold Change (capped at ±15)')
plt.ylabel('-log10 adjusted p-value')
plt.title('Volcano: ER vs TCGA')
plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))

plt.tight_layout()
plt.savefig('img/volcano_plot.png', dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
print("\nTop 10 up-regulated genes:")
print(top_up.loc[:,['gene_name']].to_string(index=False))
print("\nTop 10 down-regulated genes:")
print(top_down.loc[:,['gene_name']].to_string(index=False))

In [ ]:
# Create violin-boxplot for top 20 genes by absolute log fold change
# Sort by absolute log fold change and get top 20
top_20_genes = res_df.iloc[res_df['log2FoldChange'].abs().argsort()[::-1][:20]].index.tolist()

# Prepare data for plotting
plot_data = []
for gene in top_20_genes:
    gene_idx = np.where(df_combined.index == gene)[0][0]
    # Get expression values for cohort 1
    idx1 = 2+rnaseq_data_ER_counts.shape[1]-2
    cohort1_values = df_combined.iloc[gene_idx, 2:idx1].values
    for val in cohort1_values:
        plot_data.append({
            'Gene': gene,
            'Expression': val,
            'Group': 'Cohort 1'
        })
    # Get expression values for cohort 2
    idx2 = 2+rnaseq_data_ER_counts.shape[1]-2
    cohort2_values = df_combined.iloc[gene_idx, idx2:].values
    for val in cohort2_values:
        plot_data.append({
            'Gene': gene,
            'Expression': val,
            'Group': 'Cohort 2'
        })

plot_df = pd.DataFrame(plot_data)

# Create the violin-boxplot
plt.figure(figsize=(15, 8))
sns.set_style("whitegrid")
ax = sns.boxplot(
    data=plot_df,
    x='Gene',
    y=np.log10(plot_df['Expression'] + 1),
    hue='Group',
    #split=True,
    #inner='box',
    palette='Set2'
)

plt.xticks(rotation=45, ha='right')
plt.title('Most differentially expressed genes')
plt.xlabel('Genes')
plt.ylabel('Expression Level, log10')
plt.tight_layout()
plt.savefig('img/violin_boxplot.png')
plt.close()

## ML

### Train

In [13]:
#df_combined = pd.read_csv("data/counts_combined.csv")
df_combined = pd.read_csv("data/tpm_combined.csv")

Y = pd.read_csv("data/counts_labels.csv")

In [ ]:

df_combined_test = df_combined.iloc[0:1000,]

In [14]:
# setup & train
models = [
    SelfSupervisedML(),
    #LogisticRegressionML(),
    RandomForestML(),
    #AdaBoostML(),
    #GradientBoostML()
]

vec = [item for sublist in  Y.values.tolist() for item in sublist]

# Train and test all models
# Train all models
results_df, best_models = train_and_test_mls(
    matrix=df_combined_test,
    Y=vec,
    seed=42,
    test_size = .4,
    ml_methods=models
)

Training SelfSupervisedML:   0%|          | 0/2 [00:00<?, ?it/s]


Silhouette Score: 0.275



Silhouette Score: 0.313



Silhouette Score: 0.310



Silhouette Score: 0.315


...


Silhouette Score: 0.234



Silhouette Score: 0.234


Training RandomForestML:  50%|█████     | 1/2 [02:02<02:02, 122.64s/it]  


SelfSupervisedML best parameters:
{'hidden_dims': [512, 256, 128], 'embedding_dim': 64, 'batch_size': 32, 'learning_rate': 0.001, 'num_epochs': 100}

RandomForestML best parameters:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


Training RandomForestML: 100%|██████████| 2/2 [02:23<00:00, 71.77s/it] 


In [8]:
best_models['SelfSupervisedML'].plot_network_architecture()

# Look at predictions
print(results_df.head())

   true SelfSupervisedML LogisticRegressionML RandomForestML AdaBoostML  \
0  TCGA             TCGA                 TCGA           TCGA       TCGA   
1  TCGA             TCGA                 TCGA           TCGA       TCGA   
2  TCGA             TCGA                 TCGA           TCGA       TCGA   
3  TCGA             TCGA                 TCGA           TCGA       TCGA   
4  TCGA             TCGA                 TCGA           TCGA       TCGA   

  GradientBoostML  
0            TCGA  
1            TCGA  
2            TCGA  
3            TCGA  
4            TCGA  


In [20]:
plot_all_metrics(
    matrix=df_combined.iloc[0:100,],
    Y=vec,
    seed=42,
    results_df=results_df,
    output_folder='img/ML/'
)

/nfs/home/dsmutin/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/nfs/home/dsmutin/.local/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
# save models
import os

os.makedirs("models", exist_ok=True)

best_models['SelfSupervisedML'].save_model("models/SelfSupervisedML.pkl")
best_models['LogisticRegressionML'].save_model("models/LogisticRegressionML.pkl")
best_models['RandomForestML'].save_model("models/RandomForestML.pkl")
best_models['AdaBoostML'].save_model("models/AdaBoostML.pkl")
best_models['GradientBoostML'].save_model("models/GradientBoostML.pkl")

## Apply ML

In [ ]:
rnaseq_data_all = load_rnaseq_data_batch(
    root_dir="data/decompressed/TCGA",
    verbose=True
)

In [28]:
rnaseq_data_all[3].to_csv("data/tpm_full.csv")
rnaseq_data_all[0].to_csv("data/counts_full.csv")

In [22]:
rnaseq_data_tpm = pd.read_csv("data/tpm_full.csv")
#rnaseq_data_counts = pd.read_csv("data/counts_full.csv")

In [29]:
best_model = best_models['SelfSupervisedML']

ERvsTCGA = best_model.predict(rnaseq_data_tpm.iloc[0:1000,3:].values)
ERvsTCGA_df = pd.DataFrame(ERvsTCGA, index=rnaseq_data_tpm['gene_name'], columns=['ERvsTCGA'])
ERvsTCGA_df.value_counts()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x3352 and 1000x512)